## Initial setup

In [40]:
import os

os.chdir('/Users/cecilia/2024-winter-compostable')


In [97]:
import pandas as pd

all_sheets = pd.read_excel(
    'data/compiled_results/facility_condition/Donated Data 2023 - Compiled Facility Conditions for DSI.xlsx',
    sheet_name=None,
    skiprows=1 # most of the sheets have one empty row for condition names
)

In [98]:
# show all sheet names for later use
list(all_sheets.keys())

['Index',
 'Facility ID Correlation',
 'TrialDuration',
 ' Temperature',
 'Moisture In Field Table',
 'MoistureInFieldandLab',
 'O2 in field',
 'BD in field',
 ' BD field and lab',
 'OP12a - CN',
 'OP13a - pH',
 'Lab-Abridged-AllStages',
 'Lab-Full-SecondRemoval']

## Data cleaning for each sheet

In [108]:
def clean_column_names(column):
    column = column.lower()
    column = column.replace(' ', '_')
    column = column.replace('/', '_per_')
    column = column.replace(':', '_to_')
    for char in ['(', ')', '*', '-', '&', '?', '\n', '#', '.']:
        column = column.replace(char, '')
    
    # remove leading and trailing underscore
    column = column.strip('_')

    return column

### 1. facility_id table

In [100]:
facility_id = all_sheets['Facility ID Correlation']
facility_id.columns = [clean_column_names(col) for col in facility_id.columns]

facility_id.head()

,facility_id_cftp,trial_id_cftp,public_trial_id,trial_facility_name
0,45184,45184-01,WR004-01,Facility 1 ( Windrow)
1,50361,50361-01,CASP005-01,Facility 2 (CASP)
2,51137,51137-01,EASP001-01,Facility 3 (EASP)
3,51267,51267-01,IV002-01,Facility 4 (In-Vessel)
4,53183,53183-01,EASP002-01,Facility 5 (EASP)


### 2. trail_duration table

In [101]:
# skip empty rows (I only skip 1 row when reading the excel, but trial_duration table starts from row 3)
column_names = all_sheets['TrialDuration'].iloc[1]
trial_duration = all_sheets['TrialDuration'].loc[2:]
trial_duration.columns = column_names
trial_duration.columns = [clean_column_names(col) for col in trial_duration.columns]

trial_duration.head()

,facility_designation,midpoint_analysis,endpoint_analysis_trial_length,typical_process_from_start_to_sale,%_trial_length_to_typical_process
2,Facility 1 ( Windrow),43,67,85,0.788235
3,Facility 2 (CASP),45,71,75,0.946667
4,Facility 3 (EASP),45,72,72.5,0.993103
5,Facility 4 (In-Vessel),19,49,50,0.98
6,Facility 5 (EASP),48,93,140,0.664286


### 3. condition tables that measured daily

- Only *temperature* was measured daily.

In [102]:
temperature = all_sheets[' Temperature']
temperature.columns = temperature.columns.str.replace('*', '', regex=False) # remove * in facility name in order to better merge with other dataset

temperature.head()

,Day #,Facility 1 (Windrow),Facility 2 (CASP),Facility 3 (EASP),Facility 4 (In-Vessel),Facility 5 (EASP),Facility 6 (CASP),Facility 7 (CASP),Facility 8 (ASP),Facility 9 (EASP),Facility 10 (Windrow)
0,1,112.8,128.8,92.163333,103.625000,89.166667,103.0,172.352,127.8,120.4,146.500000
1,2,117.0,137.6,133.090000,118.250000,114.666667,NaN,169.172,133.8,116.2,136.000000
2,3,125.2,140.8,116.846667,117.875000,129.666667,109.0,164.956,136.4,124.2,141.000000
3,4,128.0,NaN,156.800000,118.428571,NaN,NaN,161.138,NaN,NaN,142.000000
4,5,NaN,NaN,161.970000,118.000000,NaN,NaN,144.674,NaN,NaN,142.333333


- As can be seen above, the current table is a wide table. 

In [133]:
temperature_long = pd.melt(temperature, id_vars='Day #', var_name='facility_name', value_name='temperature_f')
temperature_long.columns = [clean_column_names(col) for col in temperature_long.columns]
temperature_long['facility_name'] = temperature_long['facility_name'].str.split().str[0:2].str.join(' ') # standardize the name for later merge

temperature_long.head()

,day,facility_name,temperature_f
0,1,Facility 1,112.8
1,2,Facility 1,117.0
2,3,Facility 1,125.2
3,4,Facility 1,128.0
4,5,Facility 1,NaN


- Change the wide table into long one: each facility each day has an obervation for temperation, which makes it easier to merge the condition tables with mass and surface area change for the 10 trials.

### 4. condition tables that measured weekly

- In-field *Moisture* and *O2* are measured weekly.

    Although lab result observations for *moisture*, *bulk density*, *C:N ratio*, and *pH* are recorded on separate sheets for different weeks, each facility only has three observations corresponding to different trial stages. So I only keep these conditions in stage-specific tables.

In [104]:
def load_wide_transform_long(sheetname, id_vars, var_name, value_name, end_row=14):
    condition = all_sheets[sheetname].iloc[:end_row]
    condition.columns = condition.columns.str.replace('*', '', regex=False)
    condition_long = pd.melt(condition, id_vars=id_vars, var_name=var_name, value_name=value_name)
    condition_long.columns = [clean_column_names(col) for col in condition_long.columns]

    return condition_long


##### moisture_field

In [105]:
moisture_field_long = load_wide_transform_long('Moisture In Field Table', 'Week', 'facility_name', '%_moisture_field')
moisture_field_long.head()

,week,facility_name,%_moisture_field
0,1,Facility 1,0.5
1,2,Facility 1,0.5
2,3,Facility 1,0.5
3,4,Facility 1,0.5
4,5,Facility 1,0.5


##### o2_field

In [106]:
o2_field_long = load_wide_transform_long('O2 in field', 'Week', 'facility_name', '%_o2_field')
o2_field_long['week'] = o2_field_long['week'].astype(int)
o2_field_long.head()

,week,facility_name,%_o2_field
0,1,Facility 1,0.075
1,2,Facility 1,0.06
2,3,Facility 1,0.07
3,4,Facility 1,0.06
4,5,Facility 1,0.07


##### Merge the two together

In [111]:
merged_weekly_condition = pd.merge(moisture_field_long, o2_field_long, on=['week', 'facility_name'])
merged_weekly_condition.head()

,week,facility_name,%_moisture_field,%_o2_field
0,1,Facility 1,0.5,0.075
1,2,Facility 1,0.5,0.06
2,3,Facility 1,0.5,0.07
3,4,Facility 1,0.5,0.06
4,5,Facility 1,0.5,0.07


### 5. condition tables that measured at 3 stages

##### lab data for all stages

In [132]:
lab_data_all_stages = all_sheets['Lab-Abridged-AllStages']
lab_data_all_stages.columns = [clean_column_names(col) for col in lab_data_all_stages.columns]
lab_data_all_stages=lab_data_all_stages.rename(columns = {'facility_number_new':'facility_name'})

lab_data_all_stages.head()

,facility_name,trial_stage,%_total_n_as_rcvd,%_total_n_dry_wt,%_organic_c_as_rcvd,%_organic_c_dry_wt,c_to_n_dry_wt,bulk_density_lbs_per_yd_as_rcvd,%_solids_as_rcvd,%_moisture_as_rcvd,ph_as_rcvd,ph_dry_wt,stability,co2_evolution_1_mg_co2c_per_g_om_per_day,co2_evolution_2_mg_co2c_per_g_ts_per_day,overall_notes
0,Facility 1,First Removal,0.0045,0.0068,0.108,0.163,24.0,1382.0,0.665,0.335,7.6,NaN,Very Stable,0.4,0.2,NaN
1,Facility 2,First Removal,0.0096,0.0146,0.251,0.382,26.0,652.0,0.657,0.343,7.9,NaN,Stable,1.8,2.8,NaN
2,Facility 3,First Removal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45 day sample was lost by mail carrier
3,Facility 4,First Removal,0.0072,0.0135,0.224,0.421,31.0,832.0,0.532,0.468,5.0,NaN,Stable,3.8,10.9,*18 day removal
4,Facility 5,First Removal,0.0074,0.0169,0.141,0.325,19.0,963.0,0.435,0.565,6.0,NaN,Stable,3.9,8.2,NaN


In [122]:
lab_data_all_stages['trial_stage'].unique()

['First Removal', 'Loading', 'Second Removal']
Categories (3, object): ['Loading' < 'First Removal' < 'Second Removal']

- Note that loading means start.

##### more detailed lab data on the last stage

In [141]:
lab_data_detailed_end = all_sheets['Lab-Full-SecondRemoval']
lab_data_detailed_end.columns = [clean_column_names(col) for col in lab_data_detailed_end.columns]
lab_data_detailed_end=lab_data_detailed_end.rename(columns = {'facility_number_new':'facility_name',
                                                              'pass_per_fail':'pass_vs_fail',
                                                              '3_per_4__%_passing':'3_divide_4_%_passing',
                                                              '5_per_8__%_passing':'5_divide_8_%_passing',
                                                              '1_per_2__%_passing':'1_divide_2_%_passing',
                                                              '3_per_8__%_passing':'3_divide_8_%_passing',
                                                              '1_per_4__%_passing':'1_divide_4_%_passing'
                                                              }
                                                    )

# change 'end of curing' in the original data set into 'second removal' in order to merge using trial stage later
lab_data_detailed_end['trial_stage'] = lab_data_detailed_end['trial_stage'].replace('End of Curing', 'Second Removal')                       

pd.set_option('display.max_columns', None)
lab_data_detailed_end.head()

,facility_name,trial_stage,total_n_as_rcvd,total_n_dry_wt,organic_c_as_rcvd,organic_c_dry_wt,c_to_n_dry_wt,bulk_density_as_rcvd,solids_as_rcvd,moisture_as_rcvd,ph_as_rcvd,ph_dry_wt,stability_mg_co2c_per_g_om_per_day,stability_rating,emergence_%,vigor_%,maturity_rating,fecal_coliforms_mpn_per_g_dw,pass_per__fail,organic_matter_%dw,ash_%dw,ammonium_n_mg_per_kg_dw,nitraten_mg_per_kg_dw,chloride_mg_per_kg_dw,sulfates_mg_per_kg_dw,caco3_lbs_per_t,phosphorus_%dw,p2o5_%dw,potassium_%dw,k2o_%dw,calcium_%dw,magnesium_%dw,sodium_%dw,sulfur_%dw,boron_mg_per_kg_dw,zinc_mg_per_kg_dw,manganese_mg_per_kg_dw,copper__mg_per_kg_dw,iron_mg_per_kg_dw,arsenic_mg_per_kg_dw,cadmium_mg_per_kg_dw,chromium_mg_per_kg_dw,cobalt_mg_per_kg_dw,mercury_mg_per_kg_dw,molybdenum_mg_per_kg_dw,nickel_mg_per_kg_dw,lead_mg_per_kg_dw,selenium_mg_per_kg_dw,3__%_passing,2__%_passing,1__%_passing,3_divide_4_%_passing,5_divide_8_%_passing,1_divide_2_%_passing,3_divide_8_%_passing,1_divide_4_%_passing,total_plastic_%_by_wt,film_plastic_%_by_wt,glass_%_by_wt,metal_%_by_wt,sharps_%_by_wt,total_inerts_%_by_wt
0,Facility 1,Second Removal,0.432702,0.73,6.875806,11.6,15.890411,1474.178166,59.274194,40.725806,7.4,NaN,0.117282,Very Stable,100,100,Mature,388.027211,Fail,22.929724,77.070276,386.340136,4.048980,110,13.2,190,0.1031,0.236099,0.2573,0.30876,3.465,1.6300,0.0247,0.0650,1.0,74.7,460,23.8,17100,3.1,<MDL,10.4,5.0,0.02,0.5,9.6,5,0.7,100,100.000000,100.000000,100.000000,98.035139,86.225179,72.107288,48.913608,0.0,0.0,0.0,0,0,0.0
1,Facility 2,Second Removal,0.830620,1.23,18.300648,27.1,22.032520,1203.257135,67.530065,32.469935,7.1,NaN,2.256662,Stable,100,100,Mature,34.058904,Pass,42.407775,57.592225,503.479452,23.989315,2817,123.2,154,0.2498,0.572042,0.3524,0.42288,16.770,0.3964,0.2969,0.1490,43.3,28.8,117,20.7,4721,<MDL,0.2,7.1,0.8,0.01,0.8,1.7,<MDL,3.4,100,100.000000,100.000000,100.000000,99.686520,98.798328,97.405085,90.613027,0.0,0.0,0.0,0,0,0.0
2,Facility 3,Second Removal,1.116630,1.84,20.512012,33.8,18.369565,842.778000,60.686427,39.313573,7.3,NaN,1.885598,Stable,93,100,Mature,23069.408740,Fail,59.871226,40.128774,374.053985,14.006427,813,93.2,134,0.2177,0.498533,0.1965,0.23580,6.944,0.3998,0.0973,0.2486,1.6,57.9,289,23.6,4695,3.6,0.5,8.7,1.0,0.02,0.6,6.5,4.4,4.1,100,97.960159,92.972112,87.840637,83.490040,75.235060,68.031873,51.378486,0.0,0.0,0.0,0,0,0.0
3,Facility 4,Second Removal,1.323821,2.20,26.957816,44.8,20.363636,656.600678,60.173697,39.826303,6.5,NaN,3.139574,Stable,93,100,Mature,598.268041,Fail,83.246655,16.753345,789.381443,16.452371,3426,88.0,74,0.2535,0.580515,0.7345,0.88140,2.369,0.2471,0.2645,0.1566,17.2,51.3,113,34.2,2213,2.2,0.4,4.3,0.3,0.02,0.7,2.1,1.6,2.6,100,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,87.069409,0.0,0.0,0.0,0,0,0.0
4,Facility 5,Second Removal,0.450000,0.96,9.400000,19.9,21.000000,827.000000,47.100000,52.900000,8.4,NaN,1.607838,VERY STABLE,100,100,Mature,3800.000000,Fail,36.100000,63.900000,1644.900000,9.800000,1965,47.2,76,0.2100,0.470000,0.7100,0.85000,1.780,0.5100,0.1400,0.1200,9.7,76.6,303,19.3,13460,5.3,<MDL,24.7,4.6,0.03,1.5,13.8,16.9,1,100,94.140000,74.510000,66.210000,61.500000,48.640000,40.600000,26.440000,0.0,0.0,0.0,0,0,0.0
